<a href="https://colab.research.google.com/github/NaHyeonMaeng/CODE_Practice/blob/main/%EB%AC%B8%EC%9E%A5_%EC%83%9D%EC%84%B1_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# 네이버 영화리뷰 데이터셋 불러오기
file = tf.keras.utils.get_file(
    'ratings_train.txt',
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',
    extract=True)

df = pd.read_csv(file, sep='\t')

14628807/14628807 [==============================] - 0s 0us/step


In [3]:
# 데이터 임의샘플 확인
df[1000:1007]

,id,document,label
1000,9856453,정말 최고의 명작 성인이 되고 본 이집트의 왕자는 또 다른 감동 그자체네요,1
1001,6961803,이영화만 성공 했어도 스퀘어가 에닉스랑 합병 할일은 없었을텐데..,0
1002,8681713,울컥하는 사회현실 ㅠㅠ,1
1003,5348290,기대를하나도안하면 할일없을때보기좋은영화,0
1004,9340549,소림사 관문 통과하기 진짜 어렵다는거 보여준 영화..극장에서 개봉하는거 반갑다..,1
1005,7357684,시리즈안나오나 ㅠㅠㅠㅠㅠㅠㅠㅠ,1
1006,9303587,끝난다는 사실이 너무 슬퍼요. 가슴이 뻥 뚫려버린것같아..,1


데이터 전처리

In [4]:
# 형태소 분석기
!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 46.9 MB/s eta 0:00:00


In [5]:
# 데이터 전처리
def word_tokenization(text):
  return [word for word in okt.morphs(text)]

def preprocessing(df):
  df = df.dropna()
  df = df[1000:2000]  # 샘플 데이터 1000개, 학습시간을 줄이고자 함
  df['document'] = df['document'].str.replace("[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ ]","")  #영어, 한글, 띄어쓰기만 남김
  data =  df['document'].apply((lambda x: word_tokenization(x)))  #형태소 단위로 분리
  return data

In [6]:
# 텍스트 데이터 1000개 전처리 후 불러오기
review = preprocessing(df)
len(review)  #길이 확인

<ipython-input-5-65b8a2bef762>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['document'] = df['document'].str.replace("[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ ]","")  #영어, 한글, 띄어쓰기만 남김


1000

In [7]:
# 형태소 분리된 데이터 확인
print(review[:10])

1000    [정말, 최고, 의, 명작, 성인, 이, 되고, 본, 이집트, 의, 왕자, 는, 또...
1001    [이영화, 만, 성공, 했어도, 스퀘어, 가, 에, 닉스, 랑, 합병, 할, 일, ...
1002                                 [울컥, 하는, 사회, 현실, ㅠㅠ]
1003       [기대, 를, 하나, 도안, 하, 면, 할, 일, 없을, 때, 보기, 좋은, 영화]
1004    [소림사, 관문, 통과, 하기, 진짜, 어렵다는거, 보여준, 영화, 극장, 에서, ...
1005                              [시리즈, 안, 나오나, ㅠㅠㅠㅠㅠㅠㅠㅠ]
1006        [끝난다는, 사실, 이, 너무, 슬퍼요, 가슴, 이, 뻥, 뚫려, 버린것, 같아]
1007                                             [펑점, 조절]
1008                            [와이, 건, 진짜, 으리, 으리, 한, 데]
1009                                [손발, 이, 오, 그라드, 네, 요]
Name: document, dtype: object


In [8]:
# 토큰화 및 패딩 -> 단어사전 만들기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()

def get_tokens(review):
  tokenizer.fit_on_texts(review)
  total_words = len(tokenizer.word_index)+1
  tokenized_sentences = tokenizer.texts_to_sequences(review)

  input_sequences = []
  for token in tokenized_sentences:
    for t in range(1, len(token)):
        n_gram_sequence = token[:t+1]
        input_sequences.append(n_gram_sequence)

  return input_sequences, total_words

input_sequences, total_words = get_tokens(review)
input_sequences[31:40] # n_gram으로 리스트된 데이터샘플 확인

[[792, 25],
 [792, 25, 539],
 [792, 25, 539, 140],
 [792, 25, 539, 140, 109],
 [338, 9],
 [338, 9, 110],
 [338, 9, 110, 540],
 [338, 9, 110, 540, 90],
 [338, 9, 110, 540, 90, 148]]

In [10]:
# 단어 사전에 키워드 넣기
print("감동 ==>> ",tokenizer.word_index['감동'])
print("영화 ==>> ",tokenizer.word_index['영화'])
print("코믹 ==>> ",tokenizer.word_index['코믹'])

감동 ==>>  46
영화 ==>>  2
코믹 ==>>  415


In [11]:
# 문장의 길이 동일하게 맞추기
max_len = max([len(word) for word in input_sequences])
print("max_len:", max_len)
input_sequences = np.array(pad_sequences(input_sequences,
                                         maxlen=max_len,
                                         padding='pre'))  #앞에 0을 채움

max_len: 59


In [12]:
# 입력텍스트와 타겟
from tensorflow.keras.utils import to_categorical
X = input_sequences[:,:-1]  # 마지막 값은 제외함
y = to_categorical(input_sequences[:,-1],
                   num_classes=total_words) # 마지막 값만 이진 클래스 벡터로 변환

In [13]:
# y를 설명하기 위한 예시
a = to_categorical([0, 1, 2, 3], num_classes=4)
a

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

문장 생성 모델

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

embedding_dim = 256

model = Sequential([
    Embedding(input_dim=total_words,
              output_dim=embedding_dim,
              input_length=max_len-1),  #레이블 시퀀스로 한 단어를 빼서 입력 시퀀스 길이가 줄어들기 때문
    Bidirectional(LSTM(units=256)),
    Dense(units=total_words, activation='softmax'),
])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
375/375 [==============================] - 57s 118ms/step - loss: 7.7532 - accuracy: 0.0262
Epoch 2/20
375/375 [==============================] - 15s 41ms/step - loss: 7.1476 - accuracy: 0.0342
Epoch 3/20
375/375 [==============================] - 11s 28ms/step - loss: 6.7238 - accuracy: 0.0466
Epoch 4/20
375/375 [==============================] - 7s 20ms/step - loss: 6.0541 - accuracy: 0.0685
Epoch 5/20
375/375 [==============================] - 5s 13ms/step - loss: 5.1421 - accuracy: 0.1117
Epoch 6/20
375/375 [==============================] - 5s 14ms/step - loss: 4.1517 - accuracy: 0.2101
Epoch 7/20
375/375 [==============================] - 5s 13ms/step - loss: 3.2111 - accuracy: 0.3743
Epoch 8/20
375/375 [==============================] - 6s 16ms/step - loss: 2.3974 - accuracy: 0.5401
Epoch 9/20
375/375 [==============================] - 5s 13ms/step - loss: 1.7656 - accuracy: 0.6809
Epoch 10/20
375/375 [==============================] - 5s 12ms/step - loss: 1.2924 - ac

문장 생성 (추론)

In [15]:
# 문장생성함수 (시작 텍스트, 생성 단어 개수)
def text_generation(sos, count):
    for _ in range(1, count):
        token_list = tokenizer.texts_to_sequences([sos])[0]
        token_list = pad_sequences([token_list],
                                   maxlen=max_len-1,
                                   padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=1) # 최대 확률값 인덱스 반환

        for word, idx in tokenizer.word_index.items():
            if idx == predicted:
                output = word
                break
        sos += " " + output
    return sos

In [16]:
# argmax 설명: 최대값의 인덱스 반환
data = [[0.1, 0.2, 0.7], [0.3, 0.5, 0.2], [0.4, 0.3, 0.3]]
np.argmax([data], axis=-1)

array([[2, 1, 0]])

In [17]:
text_generation("연애 하면서", 12)

1/1 [==============================] - 0s 29ms/step


'연애 하면서 또 봐도 재밌고 신선한 드라마 수능 끝나면다시 봐야지 봐야지 ㅋㅋㅋ 않은데'

In [18]:
text_generation("꿀잼", 12)

1/1 [==============================] - 0s 35ms/step


'꿀잼 근데 아나킨 은 언제 지 재미 는 동시 이번 이 피'

In [19]:
text_generation("최고의 영화", 12)

1/1 [==============================] - 0s 22ms/step


'최고의 영화 정말 잘 만들었습니다 이런 영화 자주 나왔으면 좋겠습니다 좋겠습니다 도 나오고'

In [20]:
text_generation("손발 이", 12)

1/1 [==============================] - 0s 22ms/step


'손발 이 오 그라드 네 요 요 도 봤는데 도 없고 드라마 도'

In [21]:
text_generation("그냥", 12)

1/1 [==============================] - 0s 22ms/step


'그냥 쓰레기 재미 두 없구 그렇다구 그닥 야하지두 않구 ㅋ ㅋ 너무'